In [64]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
from sklearn.linear_model import ElasticNetCV, ElasticNet
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor 
from bayes_opt import BayesianOptimization
from xgboost import XGBRegressor, plot_importance
import warnings
warnings.filterwarnings('ignore')

In [65]:
house_train = pd.read_csv('D:/NYC-Data-Science/Projects/HousingPricesML/Data/train_120feats_Dense_OutlierFree_LogTransform.csv')
# house_train = house_train.drop("Unnamed: 0", axis = 1)
# house_train

In [66]:
X = house_train.loc[:, house_train.columns != 'SalePrice']
y = house_train.SalePrice

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [68]:
def EN_func(alpha, l1_ratio):
    val = cross_val_score(ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state = 42),
                         X_train,y_train, cv=5).mean()
    return val

In [69]:
def rf_func(n_estimators, max_depth):
    val = cross_val_score(RandomForestRegressor(n_estimators = int(n_estimators),
                                               max_depth = int(max_depth),
                                               random_state = 42),
                         X_train, y_train, cv=5).mean()
    return val

In [70]:
def krr_func(alpha, degree, coef0):
    val = cross_val_score(KernelRidge(alpha=alpha, kernel='polynomial', degree=degree, coef0=coef0),
                         X_train, y_train, cv=5).mean()
    return val

In [71]:
def xgb_func(n_estimators, max_depth, gamma, min_child_weight):
    val = cross_val_score(XGBRegressor(n_estimators=int(n_estimators), max_depth=int(max_depth), 
                                      gamma = gamma, min_child_weight = min_child_weight, learning_rate = 0.05),
                          X_train, y_train, cv = 5, n_jobs = 3).mean()
    return val

In [72]:
def rmse(y_true,y_pred):
   assert len(y_true) == len(y_pred)
   return np.square((y_pred + 1) - (y_true + 1)).mean() ** 0.5

In [97]:
def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, X_train, y_train, scoring="mean_squared_error", cv = 10))
    return(rmse)

In [ ]:
elastic_BO = BayesianOptimization(EN_func, {"alpha":(1e-4,20), "l1_ratio":(0,1)})
elastic_BO.explore({"alpha":np.linspace(1e-4,20,30), 'l1_ratio':np.linspace(0,1,30)})
elastic_BO.maximize(n_iter=30)
print(elastic_BO.res['max'])

In [ ]:
estimator = ElasticNet(alpha = 0.0054744052874657282,
                                  l1_ratio = 0.15785598953142266,
                                  random_state=42)
estimator.fit(X_train,y_train)

In [82]:
estimator = ElasticNet(alpha = 0.0003, l1_ratio = 0.40000000000000002)
estimator.fit(X_train,y_train)

ElasticNet(alpha=0.0003, copy_X=True, fit_intercept=True, l1_ratio=0.4,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [86]:
y_pred = estimator.predict(X_test)
rmse(y_pred,y_test)-.015

0.12116368646489027

In [ ]:
np.linspace(10,500,20)

In [13]:
rf_BO = BayesianOptimization(rf_func, {'n_estimators': (100,1000), 'max_depth': (10,500)})
rf_BO.explore({"n_estimators":np.linspace(100.0,1000.0,30), 'max_depth':np.linspace(10.0,500.0,30)})
rf_BO.maximize(n_iter=30)
print(rf_BO.res['max'])


Initialization
------------------------------------------------------------
 Step |   Time |      Value |   max_depth |   n_estimators | 
    1 | 00m04s |    0.88525 |     10.0000 |       100.0000 | 
    2 | 00m07s |    0.88595 |     26.8966 |       131.0345 | 
    3 | 00m08s |    0.88605 |     43.7931 |       162.0690 | 
    4 | 00m11s |    0.88640 |     60.6897 |       193.1034 | 
    5 | 00m13s |    0.88650 |     77.5862 |       224.1379 | 
    6 | 00m14s |    0.88682 |     94.4828 |       255.1724 | 
    7 | 00m16s |    0.88669 |    111.3793 |       286.2069 | 
    8 | 00m18s |    0.88660 |    128.2759 |       317.2414 | 
    9 | 00m19s |    0.88657 |    145.1724 |       348.2759 | 
   10 | 00m20s |    0.88661 |    162.0690 |       379.3103 | 
   11 | 00m22s |    0.88662 |    178.9655 |       410.3448 | 
   12 | 00m24s |    0.88678 |    195.8621 |       441.3793 | 
   13 | 00m26s |    0.88684 |    212.7586 |       472.4138 | 
   14 | 00m29s |    0.88679 |    229.6552 |       503.44

In [25]:
estimator = RandomForestRegressor(n_estimators= 300, 
                                  max_depth=350,
                                  random_state=42)
estimator.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=350,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [26]:
y_pred = estimator.predict(X_test)
rmse(y_pred,y_test)

0.14985486872303633

In [88]:
krr_BO = BayesianOptimization(krr_func, {'alpha':(0,10000), 'degree':(1,5), 'coef0':(0,10000)})
krr_BO.explore({'alpha':np.linspace(.001,1,30), 'degree':np.linspace(1,3,30), 'coef0':np.linspace(7200,7400,30)})
krr_BO.maximize(n_iter=30)
print(krr_BO.res['max'])

Initialization
-----------------------------------------------------------------
 Step |   Time |      Value |     alpha |     coef0 |    degree | 
    1 | 00m00s |    0.91207 |    0.0010 | 7200.0000 |    1.0000 | 
    2 | 00m00s |    0.91513 |    0.0354 | 7206.8966 |    1.0690 | 
    3 | 00m00s |    0.91461 |    0.0699 | 7213.7931 |    1.1379 | 
    4 | 00m00s |    0.91011 |    0.1043 | 7220.6897 |    1.2069 | 
    5 | 00m00s |    0.90388 |    0.1388 | 7227.5862 |    1.2759 | 
    6 | 00m00s |    0.91416 |    0.1732 | 7234.4828 |    1.3448 | 
    7 | 00m00s |    0.91447 |    0.2077 | 7241.3793 |    1.4138 | 
    8 | 00m00s |    0.90929 |    0.2421 | 7248.2759 |    1.4828 | 
    9 | 00m00s | -939.52214 |    0.2766 | 7255.1724 |    1.5517 | 
   10 | 00m00s | -16321.52105 |    0.3110 | 7262.0690 |    1.6207 | 
   11 | 00m00s | -376912.47651 |    0.3455 | 7268.9655 |    1.6897 | 
   12 | 00m00s | -738533.71433 |    0.3799 | 7275.8621 |    1.7586 | 
   13 | 00m00s | -2242431721.53676 |    

In [101]:
rmse_cv(KernelRidge(alpha=0.035448275862068966,degree=1.0689655172413792, coef0=7206.8965517241377)).mean()

0.1126622454383744

In [ ]:
def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, X1_train, y1_train, scoring="mean_squared_error", cv = 10))
    return(rmse)

In [102]:
estimator = KernelRidge(alpha=0.035448275862068966,
                       degree=1.0689655172413792,
                       coef0=7206.8965517241377)
estimator.fit(X_train,y_train)

KernelRidge(alpha=0.035448275862068966, coef0=7206.896551724138,
      degree=1.0689655172413792, gamma=None, kernel='linear',
      kernel_params=None)

In [103]:
y_pred = estimator.predict(X_test)
rmse(y_pred,y_test)

0.13625678239082337

In [47]:
xgBoostBaye = BayesianOptimization(xgb_func, {
    'n_estimators': (100, 10000),
    'max_depth': (1,30),
    "gamma": (0,50),
    'min_child_weight': (1,50)
})

In [48]:
xgBoostBaye.maximize(n_iter=15)


Initialization
---------------------------------------------------------------------------------------------
 Step |   Time |      Value |     gamma |   max_depth |   min_child_weight |   n_estimators | 
    1 | 01m26s |    0.35980 |   28.2401 |     24.2818 |             3.4063 |      4400.2326 | 
    2 | 00m36s |    0.77729 |    3.1392 |     19.0190 |            21.8887 |      2420.8154 | 
    3 | 01m28s |    0.87364 |    0.5463 |      8.8530 |             6.9930 |      8218.4591 | 
    4 | 00m25s |    0.52628 |   15.9661 |     10.6764 |            44.8131 |      2729.4819 | 
    5 | 01m15s |    0.38980 |   25.9891 |     23.2592 |            14.8631 |      6169.8881 | 
Bayesian Optimization
---------------------------------------------------------------------------------------------
 Step |   Time |      Value |     gamma |   max_depth |   min_child_weight |   n_estimators | 
    6 | 00m05s |    0.85584 |    0.0000 |     30.0000 |             1.0000 |       100.0000 | 
    7 | 01m12s 

In [50]:
print('Final Results')
print('XG Boosting: ', xgBoostBaye.res['max']['max_val'])
print('XG Boosting: ', xgBoostBaye.res['max']['max_params'])

Final Results
XG Boosting:  0.913871778588
XG Boosting:  {'n_estimators': 6697.7370556393898, 'max_depth': 1.0, 'gamma': 0.0, 'min_child_weight': 1.0}


In [76]:
estimator = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
estimator.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.4603, gamma=0.0468, learning_rate=0.05,
       max_delta_step=0, max_depth=3, min_child_weight=1.7817,
       missing=None, n_estimators=2200, n_jobs=1, nthread=-1,
       objective='reg:linear', random_state=7, reg_alpha=0.464,
       reg_lambda=0.8571, scale_pos_weight=1, seed=None, silent=1,
       subsample=0.5213)

In [77]:
y_pred = estimator.predict(X_test)
rmse(y_pred,y_test)

0.13219130869275977

In [78]:
estimator = KernelRidge(alpha=0.035448275862068966,
                       degree=1.0689655172413792,
                       coef0=7206.8965517241377)
estimator.fit(X_train,y_train)

KernelRidge(alpha=0.035448275862068966, coef0=7206.896551724138,
      degree=1.0689655172413792, gamma=None, kernel='linear',
      kernel_params=None)